In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
#Heatmap (top zones × hour-of-day)


zone_hour_pd = pd.read_parquet(OUT_ZONE)                 # columns: zone, hour, trips
zone_hour_pd['hod'] = pd.to_datetime(zone_hour_pd['hour']).dt.hour
TOP_N = 20
top_z = zone_hour_pd.groupby('zone')['trips'].sum().nlargest(TOP_N).index
hm = (zone_hour_pd[zone_hour_pd['zone'].isin(top_z)]
      .groupby(['zone','hod'])['trips'].sum().unstack(fill_value=0))

plt.figure(figsize=(13,6))
plt.imshow(hm.values, aspect='auto', interpolation='nearest')
plt.colorbar(label='Trips')
plt.yticks(range(len(hm.index)), hm.index)
plt.xticks(range(24), range(24))
plt.title(f"Top {TOP_N} Zones — Trips by Hour of Day")
plt.xlabel("Hour of Day"); plt.ylabel("Zone (LocationID)")
plt.tight_layout()

In [ ]:
#Weather vs trips (citywide hourly)

city = pd.read_parquet(OUT_WXJ)                      # columns: hour, trips, prcp, temp
city['hour'] = pd.to_datetime(city['hour'])
city = city.sort_values('hour').set_index('hour')
city['trips_ma7d'] = city['trips'].rolling('7D').mean()

fig, ax1 = plt.subplots(figsize=(13,4))
city['trips_ma7d'].plot(ax=ax1)
ax1.set_ylabel("Trips (7d MA)")
ax1.set_title("Citywide Demand vs Precipitation (Hourly)")

ax2 = ax1.twinx()
city['prcp'].plot(ax=ax2, alpha=0.4)
ax2.set_ylabel("Precipitation (mm)")
fig.tight_layout()
